In [68]:
# tokenization
import gensim
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize,RegexpTokenizer
from nltk.corpus import stopwords
from collections import Counter
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.cluster.util import cosine_distance
import math
import numpy as np
import operator
import networkx as nx
nltk.download('punkt')
nltk.download('stopwords')
stop_words = stopwords.words('indonesian')
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ezifakhrezi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ezifakhrezi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Input Query dan Target File

In [69]:
query = "Mati"
target_file = 'mati.txt' # diesuaikan dengan target
document_limit = 15

In [70]:
def get_similarity(sentences, query):
    cosine_mat=np.zeros(len(sentences)+1)
   
    vector_query = text_to_vector(query) # calculate query vector
    print("vector_query : {}".format(vector_query))
    row=0
    
    # calculate cosine value for every sentence 
    for sentence in sentences:
        maxi=0
        
        vector_sentence = text_to_vector(sentence) # get vector for the sentence
        print("vector_sentence : {}".format(vector_sentence))
        
        cosine = get_cosine(vector_sentence, vector_query) # calculate cosine similarity
        print("cosine : {}".format(cosine))
        
        # prevent from negative value
        if(maxi<cosine):
            maxi=cosine
               
        cosine_mat[row]=maxi # set to sentence index
             
        row+=1
    print("cosine_mat : {}".format(cosine_mat))    
    similarity = sum(cosine_mat) # sum all cosine array
    print("similarity : {}".format(similarity)) 
    
    return similarity

In [71]:
def text_to_vector(text):
    words = preprocessing(text)
    return Counter(words)

In [72]:
# word to vector
def get_cosine(vec1, vec2):
    
    intersection = set(vec1) & set(vec2.keys()) # find the same word
    print("Intersection : {}".format(intersection))
    
    numerator = sum([vec1[x] * vec2[x] for x in intersection]) # iterate intersection, and total the times frequency
    print("numerator : {}".format(numerator))
    
    sum1 = sum([vec1[x]**2 for x in vec1.keys()]) # iterate the dictionary, and square the elements then sum it
    print("sum1 : {}".format(sum1))
    
    sum2 = sum([vec2[x]**2 for x in vec2.keys()]) # iterate the vector2 and square the elements then sum it
    print("sum2 : {}".format(sum2))
    
    denominator = math.sqrt(sum1) * math.sqrt(sum2) # square root the sum1 times square root the sum2
    print("den : {}".format(sum2))
    
    if not denominator:
        # the denominator is zero
        return 0.0
    else:
        return numerator / denominator

In [73]:
def preprocessing(data):
    """
    Do pre processing data, tokenization, remove stop words, and stemming
    Input : data (string)
    Output : array of words
    """
    #tokenizer = RegexpTokenizer(r'\w+') # allow charachter only
    #words = tokenizer.tokenize(data) # tokenize : convert to words
    words = word_tokenize(data)
    # remove stop words & stemming
    new_words = []
    print("Words : {}".format(words))
    for word in words:
        if word not in stop_words:
            new_words.append(stemmer.stem(word)) # append to new words with stemming
    
    if '' in new_words: new_words.remove('')
    print("Preprocessing : {}".format(new_words))
    
    return new_words

In [74]:
def read_article_2(filename):
    file = open(filename, "r")
    filedata = file.readlines()
    sentences = sent_tokenize(filedata[0])
    return sentences

In [75]:
import os
folder_root = 'data set'
sub_folder = 'Tafsir al-Misbah'
directory_path = os.path.join(folder_root,sub_folder) # join folder root and label
files = sorted(os.listdir(directory_path)) # get file list with sorted

files

['.DS_Store',
 '1-6-1.txt',
 '1-6-10.txt',
 '1-6-2.txt',
 '1-6-3.txt',
 '1-6-4.txt',
 '1-6-5.txt',
 '1-6-6.txt',
 '1-6-7.txt',
 '1-6-8.txt',
 '1-6-9.txt',
 '103-1-1.txt',
 '103-1-10.txt',
 '103-1-2.txt',
 '103-1-3.txt',
 '103-1-4.txt',
 '103-1-5.txt',
 '103-1-6.txt',
 '103-1-7.txt',
 '103-1-8.txt',
 '103-1-9.txt',
 '103-3-1.txt',
 '103-3-10.txt',
 '103-3-2.txt',
 '103-3-3.txt',
 '103-3-4.txt',
 '103-3-5.txt',
 '103-3-6.txt',
 '103-3-7.txt',
 '103-3-8.txt',
 '103-3-9.txt',
 '106-1-1.txt',
 '106-1-10.txt',
 '106-1-2.txt',
 '106-1-3.txt',
 '106-1-4.txt',
 '106-1-5.txt',
 '106-1-6.txt',
 '106-1-7.txt',
 '106-1-8.txt',
 '106-1-9.txt',
 '114-1-1.txt',
 '114-1-10.txt',
 '114-1-2.txt',
 '114-1-3.txt',
 '114-1-4.txt',
 '114-1-5.txt',
 '114-1-6.txt',
 '114-1-7.txt',
 '114-1-8.txt',
 '114-1-9.txt',
 '133-5-1.txt',
 '133-5-10.txt',
 '133-5-2.txt',
 '133-5-3.txt',
 '133-5-4.txt',
 '133-5-5.txt',
 '133-5-6.txt',
 '133-5-7.txt',
 '133-5-8.txt',
 '133-5-9.txt',
 '15-66-1.txt',
 '15-66-2.txt',
 '15-66-

In [76]:
print('\n --- --- ---')
print("Query : {}".format(query))
# iterate for all files
list_similarity = {}
for file in files:
    # check for .txt only
    print('\n')
    if file.lower().endswith('.txt'):
        # read article
        file_name = os.path.join(directory_path,file) # get full path
        sentences = read_article_2(file_name) # read file
        similarity = get_similarity(sentences,query)
            
        # check if any similarity
        if (similarity > 0.0):
            list_similarity[file] = similarity # set similarity

        # check if length list_similarity more than 15, then break instead check all file 
        if len(list_similarity) >= document_limit:
            break # break
    
    
#print("Similarity : \n {}".format(list_similarity))
sorted_similarity = dict(sorted(list_similarity.items(), key=operator.itemgetter(1),reverse=True)) # sorted
print("\n Sorted Similarity : \n {}".format(sorted_similarity)) 


 --- --- ---
Query : Mati




Words : ['Mati']
Preprocessing : ['mati']
vector_query : Counter({'mati': 1})
Words : ['Kata', 'ash-', 'shirath', 'terambil', 'dari', 'kata', 'saratha', ',', 'dan', 'karena', 'huruf', 'Sin', 'dalam', 'kata', 'ini', 'bergandengan', 'dengan', 'huruf', ',', 'huruf', 'Sin', 'terucapkan', 'shad', 'shirath', 'atau', 'zai', 'zirat', '.']
Preprocessing : ['kata', 'ash-', 'shirath', 'ambil', 'saratha', 'huruf', 'sin', 'gandeng', 'huruf', '', 'huruf', 'sin', 'ucap', 'shad', 'shirath', 'zai', 'zirat', '']
vector_sentence : Counter({'huruf': 3, 'shirath': 2, 'sin': 2, '': 2, 'kata': 1, 'ash-': 1, 'ambil': 1, 'saratha': 1, 'gandeng': 1, 'ucap': 1, 'shad': 1, 'zai': 1, 'zirat': 1})
Intersection : set()
numerator : 0
sum1 : 30
sum2 : 1
den : 1
cosine : 0.0
Words : ['Asal', 'katanya', 'sendiri', 'bermakna', 'menelan', '.']
Preprocessing : ['asal', 'makna', 'tel']
vector_sentence : Counter({'asal': 1, 'makna': 1, 'tel': 1})
Intersection : set()
numerator : 0
sum1 : 3
sum2

Preprocessing : ['tak', 'anda', 'telusur', 'sabil', 'anda', 'temu', 'jalan', 'tol', 'jalan', 'luas', 'iagi', 'lurus', '']
vector_sentence : Counter({'anda': 2, 'jalan': 2, 'tak': 1, 'telusur': 1, 'sabil': 1, 'temu': 1, 'tol': 1, 'luas': 1, 'iagi': 1, 'lurus': 1, '': 1})
Intersection : set()
numerator : 0
sum1 : 17
sum2 : 1
den : 1
cosine : 0.0
Words : ['Bila', 'memasukinya', 'Anda', 'telah', 'ditelan', 'olehnya', 'dan', 'tidak', 'dapat', 'keluar', 'kecuali', 'setelah', 'tiba', 'pada', 'akhir', 'tujuan', 'perjalanan', '.']
Preprocessing : ['bila', 'masuk', 'anda', 'tel', 'kecuali', 'tuju', 'jalan']
vector_sentence : Counter({'bila': 1, 'masuk': 1, 'anda': 1, 'tel': 1, 'kecuali': 1, 'tuju': 1, 'jalan': 1})
Intersection : set()
numerator : 0
sum1 : 7
sum2 : 1
den : 1
cosine : 0.0
cosine_mat : [0. 0. 0. 0. 0.]
similarity : 0.0


Words : ['Mati']
Preprocessing : ['mati']
vector_query : Counter({'mati': 1})
Words : ['Shirath', 'adalah', 'jalan', 'yang', 'luas', ',', 'semua', 'orang', 'dapat'

Preprocessing : ['dapat', 'tiga', 'ashr', 'hadir', 'nabi', 'muhammd', 'saw', 'pentas', 'hidup']
vector_sentence : Counter({'dapat': 1, 'tiga': 1, 'ashr': 1, 'hadir': 1, 'nabi': 1, 'muhammd': 1, 'saw': 1, 'pentas': 1, 'hidup': 1})
Intersection : set()
numerator : 0
sum1 : 9
sum2 : 1
den : 1
cosine : 0.0
Words : ['Pendapat', 'yang', 'paling', 'tepat—hemat', 'penulis—adalah', 'waktu', 'secara', 'umum', '.']
Preprocessing : ['dapat', 'tepat hemat', 'tulis adalah']
vector_sentence : Counter({'dapat': 1, 'tepat hemat': 1, 'tulis adalah': 1})
Intersection : set()
numerator : 0
sum1 : 3
sum2 : 1
den : 1
cosine : 0.0
cosine_mat : [0. 0. 0. 0.]
similarity : 0.0


Words : ['Mati']
Preprocessing : ['mati']
vector_query : Counter({'mati': 1})
Words : ['Dapat', 'juga', 'dikatakan', 'bahwa', 'pada', 'surah', 'ini', 'Allah', 'bersumpah', 'demi', 'waktu', 'dan', 'dengan', 'menggunakan', 'kata', "'ashr", 'bukan', 'selainnya—untuk', 'menyatakan', 'bahwa', ':', 'Demi', 'waktu', '(', 'masa', ')', 'di', 'ma

Preprocessing : ['kata', 'quraisy', 'gelar', 'an-nadhr', 'ibn', 'kinanah', 'kakek', 'nabi', 'tiga', 'belas', '']
vector_sentence : Counter({'kata': 1, 'quraisy': 1, 'gelar': 1, 'an-nadhr': 1, 'ibn': 1, 'kinanah': 1, 'kakek': 1, 'nabi': 1, 'tiga': 1, 'belas': 1, '': 1})
Intersection : set()
numerator : 0
sum1 : 11
sum2 : 1
den : 1
cosine : 0.0
cosine_mat : [0. 0.]
similarity : 0.0


Words : ['Mati']
Preprocessing : ['mati']
vector_query : Counter({'mati': 1})
Words : ['Pendapat', 'lain', 'menyatakan', 'bahwa', 'quraisy', 'terambil', 'dari', 'kata', 'qirsy', 'yakni', 'ikan', 'Hiu', '.']
Preprocessing : ['dapat', 'quraisy', 'ambil', 'qirsy', 'ikan', 'hiu']
vector_sentence : Counter({'dapat': 1, 'quraisy': 1, 'ambil': 1, 'qirsy': 1, 'ikan': 1, 'hiu': 1})
Intersection : set()
numerator : 0
sum1 : 6
sum2 : 1
den : 1
cosine : 0.0
Words : ['Ikan', 'ini', 'sangat', 'kuat', ',', 'mengatasi', 'ikan-ikan', 'lainnya', ',', 'bahkan', 'dapat', 'menjungkirbalikkan', 'perahu-perahu', 'dan', 'menerkam',

Preprocessing : ['an-nas', 'ambil', 'an-nauws', 'gerak', 'dapat', 'ambil', 'unas', 'akar', '']
vector_sentence : Counter({'ambil': 2, 'an-nas': 1, 'an-nauws': 1, 'gerak': 1, 'dapat': 1, 'unas': 1, 'akar': 1, '': 1})
Intersection : set()
numerator : 0
sum1 : 11
sum2 : 1
den : 1
cosine : 0.0
cosine_mat : [0. 0.]
similarity : 0.0


Words : ['Mati']
Preprocessing : ['mati']
vector_query : Counter({'mati': 1})
Words : ['Kata', 'an-nas', 'digunakan', "al-Qur'an", 'dalam', 'arti', 'jenis', 'manusia', '(', 'QS', '.']
Preprocessing : ['kata', 'an-nas', 'al-qur an', 'arti', 'jenis', 'manusia', 'qs', '']
vector_sentence : Counter({'kata': 1, 'an-nas': 1, 'al-qur an': 1, 'arti': 1, 'jenis': 1, 'manusia': 1, 'qs': 1, '': 1})
Intersection : set()
numerator : 0
sum1 : 8
sum2 : 1
den : 1
cosine : 0.0
Words : ['al-Hujurat', '[', '491', ':', '13', ')', 'atau', 'sekelompok', 'tertentu', 'dari', 'manusia', '(', 'seperti', 'QS', '.']
Preprocessing : ['al-hujurat', '491', '', '13', '', 'kelompok', 'manusia'

...

Preprocessing : ['ada', 'arti', 'kaga', 'al-yamin', 'bahagia']
vector_sentence : Counter({'ada': 1, 'arti': 1, 'kaga': 1, 'al-yamin': 1, 'bahagia': 1})
Intersection : set()
numerator : 0
sum1 : 5
sum2 : 1
den : 1
cosine : 0.0
cosine_mat : [0. 0.]
similarity : 0.0


Words : ['Mati']
Preprocessing : ['mati']
vector_query : Counter({'mati': 1})
Words : ['Ulama', 'lain', 'al-yamin', 'diartikan', 'dengan', 'keberkatan', '.']
Preprocessing : ['ulama', 'al-yamin', 'arti', 'berkat']
vector_sentence : Counter({'ulama': 1, 'al-yamin': 1, 'arti': 1, 'berkat': 1})
Intersection : set()
numerator : 0
sum1 : 4
sum2 : 1
den : 1
cosine : 0.0
Words : ['Ada', 'juga', 'yang', 'mengartikan', 'al-yamin', 'sebagai', 'bukti', 'yang', 'jelas', '.']
Preprocessing : ['ada', 'arti', 'al-yamin', 'bukti']
vector_sentence : Counter({'ada': 1, 'arti': 1, 'al-yamin': 1, 'bukti': 1})
Intersection : set()
numerator : 0
sum1 : 4
sum2 : 1
den : 1
cosine : 0.0
cosine_mat : [0. 0. 0.]
similarity : 0.0


Words : ['Mati']
Pre

Preprocessing : ['kata', 'al-abrar', 'bentuk', 'jamak', 'barr', 'barr']
vector_sentence : Counter({'barr': 2, 'kata': 1, 'al-abrar': 1, 'bentuk': 1, 'jamak': 1})
Intersection : set()
numerator : 0
sum1 : 8
sum2 : 1
den : 1
cosine : 0.0
cosine_mat : [0. 0.]
similarity : 0.0


Words : ['Mati']
Preprocessing : ['mati']
vector_query : Counter({'mati': 1})
Words : ['Al-Biqâ', "'", 'i', 'melukiskan', 'al-abrar', 'adalah', 'sebagai', 'orang-orang', 'yang', 'memiliki', 'kepribadian', 'dan', 'semangat', 'yang', 'tinggi', 'sehingga', 'tidak', 'menoleh', 'kepada', 'hal-', 'hal', 'yang', 'bernilai', 'rendah', 'sehinga', 'tampak', 'dalam', 'jiwa', 'mereka', 'sumber-sumber', 'hikmah', '.']
Preprocessing : ['al-biq', 'i', 'lukis', 'al-abrar', 'orang', 'milik', 'pribadi', 'semangat', 'toleh', 'hal-', 'nila', 'rendah', 'sehinga', 'jiwa', 'sumber', 'hikmah', '']
vector_sentence : Counter({'al-biq': 1, 'i': 1, 'lukis': 1, 'al-abrar': 1, 'orang': 1, 'milik': 1, 'pribadi': 1, 'semangat': 1, 'toleh': 1, 'ha

Preprocessing : ['al-biqa', 'i', 'oleh', 'kesan', 'fi di', 'firman', 'rangkai', 'zhilalin naung', 'ada', 'sana', 'mantap', 'mantap', 'wadah', '', 'sila', 'makan', '']
vector_sentence : Counter({'mantap': 2, '': 2, 'al-biqa': 1, 'i': 1, 'oleh': 1, 'kesan': 1, 'fi di': 1, 'firman': 1, 'rangkai': 1, 'zhilalin naung': 1, 'ada': 1, 'sana': 1, 'wadah': 1, 'sila': 1, 'makan': 1})
Intersection : set()
numerator : 0
sum1 : 21
sum2 : 1
den : 1
cosine : 0.0
Words : ['Ini', 'mengisyaratkan', 'bahwa', 'yang', 'dimaksud', 'fakihah', 'bukan', 'sekadar', 'buah-buahan', 'tetapi', 'mencakup', 'segala', 'macam', 'makanan', 'yang', 'lezat', 'dan', 'menyenangkan', '.']
Preprocessing : ['ini', 'isyarat', 'fakihah', 'buah', 'cakup', 'makan', 'lezat', 'senang']
vector_sentence : Counter({'ini': 1, 'isyarat': 1, 'fakihah': 1, 'buah': 1, 'cakup': 1, 'makan': 1, 'lezat': 1, 'senang': 1})
Intersection : set()
numerator : 0
sum1 : 8
sum2 : 1
den : 1
cosine : 0.0
cosine_mat : [0. 0. 0.]
similarity : 0.0


Words : [

Preprocessing : ['bukan', 'itu ketika', 'mekanisme', 'cipta', 'allah', 'manusia', 'instruksi', 'benak', 'singkir', 'ganggu', 'tidur', 'mimpi', '', 'sangkut', 'cekik', 'bangun', 'singkir', 'bantal', '--', '-dalam', 'mimpi bahwa', 'kamar', 'basah', 'kasur', '', 'tidur', '']
vector_sentence : Counter({'': 3, 'singkir': 2, 'tidur': 2, 'bukan': 1, 'itu ketika': 1, 'mekanisme': 1, 'cipta': 1, 'allah': 1, 'manusia': 1, 'instruksi': 1, 'benak': 1, 'ganggu': 1, 'mimpi': 1, 'sangkut': 1, 'cekik': 1, 'bangun': 1, 'bantal': 1, '--': 1, '-dalam': 1, 'mimpi bahwa': 1, 'kamar': 1, 'basah': 1, 'kasur': 1})
Intersection : set()
numerator : 0
sum1 : 37
sum2 : 1
den : 1
cosine : 0.0
Words : ['Namun', 'demikian', ',', 'kita', 'pun', 'hendaknya', 'mengakui', 'bahwa', 'penyebab', 'mimpi', 'bukan', 'hanya', 'yang', 'diutarakan', 'di', 'atas', 'tetapi', 'seperti', 'sabda', 'Rasul', 'saw', ':', '``', 'Mimpi', 'ada', 'tiga', 'macam', ',', 'berita', 'gembira', 'dari', 'Allah', 'Yang', 'Maha', 'Pengasih', ',', 'b

Preprocessing : ['itu', 'nama', 'ta', 'bir', 'ar-ru ya', 'tafsir', 'singkap', 'makna', 'mimpi', '']
vector_sentence : Counter({'itu': 1, 'nama': 1, 'ta': 1, 'bir': 1, 'ar-ru ya': 1, 'tafsir': 1, 'singkap': 1, 'makna': 1, 'mimpi': 1, '': 1})
Intersection : set()
numerator : 0
sum1 : 10
sum2 : 1
den : 1
cosine : 0.0
Words : ['Itu', 'pula', 'sebabnya', 'sehingga', 'tingkat', 'terendah', 'dari', 'informasi', 'Allah', 'adalah', 'mimpi', '.']
Preprocessing : ['itu', 'tingkat', 'rendah', 'informasi', 'allah', 'mimpi']
vector_sentence : Counter({'itu': 1, 'tingkat': 1, 'rendah': 1, 'informasi': 1, 'allah': 1, 'mimpi': 1})
Intersection : set()
numerator : 0
sum1 : 6
sum2 : 1
den : 1
cosine : 0.0
cosine_mat : [0. 0. 0.]
similarity : 0.0


Words : ['Mati']
Preprocessing : ['mati']
vector_query : Counter({'mati': 1})
Words : ['Uraian', "al-Qur'an", 'tentang', 'mimpi', 'yang', 'dialami', 'Nabi', 'saw', '.']
Preprocessing : ['urai', 'al-qur an', 'mimpi', 'alami', 'nabi', 'saw']
vector_sentence : Cou

In [77]:
sorted_similarity = dict(sorted(list_similarity.items(), key=operator.itemgetter(1),reverse=True)) # sorting
sorted_similarity

{'67-2-10.txt': 0.7132007163556104,
 '67-2-7.txt': 0.696923425058676,
 '67-2-5.txt': 0.4472135954999579,
 '67-2-6.txt': 0.4472135954999579,
 '67-2-9.txt': 0.3481553119113957,
 '67-2-4.txt': 0.3333333333333333,
 '67-2-3.txt': 0.30151134457776363,
 '2-132-2.txt': 0.2886751345948129,
 '67-2-8.txt': 0.2672612419124244,
 '75-34-9.txt': 0.24253562503633297,
 '75-34-10.txt': 0.1643989873053573}

## Text Sumarization

In [78]:
def read_article(file_name):
    """
    Generate clean sentences from the file
    Input : path file name
    Output : array of sentences
    """
    
    file = open(file_name, "r")
    filedata = file.readlines()
    sentences = []   
    for sentence in filedata:
        print("Sentence : {}".format(sentence))
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" ")) # filter charachter only
    
    return sentences

In [79]:
def merge_sentences(sentences):
    """
    Merge sentences to one array
    """
    full_sentences = []
    for sentence in sentences:
        for arr_word in sentence:
            full_sentences.append(arr_word)
    return full_sentences

In [80]:
def sentence_similarity(sent1, sent2, stopwords=None):
    # preprocessing stopword
    if stopwords is None:
        stopwords = []
        #print("stopwords: {}".format(stopwords))
 
    # lower case
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    # gabungin
    all_words = list(set(sent1 + sent2))
    #print("all_words: {}".format(all_words))
    
    # word to vect
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
        #print("vector1: {}".format(vector1))
        
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
        #print("vector2: {}".format(vector2))
    return 1 - cosine_distance(vector1, vector2)

def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    
    return similarity_matrix

In [81]:
documents = []
list_file = []
for key in sorted_similarity:
    file_name = os.path.join(directory_path,key) # get full path
                
    # read file
    document = read_article(file_name)
    documents.append(document)

    # append to file list for log
    list_file.append(key)

Sentence : Ada juga yang memahaminya dalam arti: “Allah menciptakan kematian untuk membangkitkan dan memberi kamu balasan dan menciptakan kehidupan untuk menguji kamu." Atau Allah menciptakan kematian dan kehidupan untuk menguji kamu siapa yang lebih mempersiapkan diri menghadapi kematian dan siapa yang lebih bergegas memenuhi ketaatan kepada Allah.

Sentence : Ada juga yang memahami mati dalam arti ketiadaan wujud. Yang memahami demikian, memahami ayat di atas dalam arti Allah menciptakan sebab-sebab kematian. 
Sentence : Kematian manusia dalam pentas bumi ini bukanlah ketiadaan.
Sentence : Ia masih wujud tetapi berpindah ke alam lain. Itulah salah satu yang diisyaratkan oleh kata menciptakan kematian.
Sentence : Ujian menyangkut hidup dan mati dipahami oleh sementara ulama dalam arti musibah kematian yang menimpa keluarga atau teman seseorang, demikian juga anugerah kehidupan serta kelahiran merupakan bahan ujian Allah kepada manusia, apakah dia tabah dan sabar serta bersyukur dan be

In [82]:
list_file

['67-2-10.txt',
 '67-2-7.txt',
 '67-2-5.txt',
 '67-2-6.txt',
 '67-2-9.txt',
 '67-2-4.txt',
 '67-2-3.txt',
 '2-132-2.txt',
 '67-2-8.txt',
 '75-34-9.txt',
 '75-34-10.txt']

In [83]:
full_sentences = merge_sentences(documents)
full_sentences

[['Ada',
  'juga',
  'yang',
  'memahaminya',
  'dalam',
  'arti:',
  '“Allah',
  'menciptakan',
  'kematian',
  'untuk',
  'membangkitkan',
  'dan',
  'memberi',
  'kamu',
  'balasan',
  'dan',
  'menciptakan',
  'kehidupan',
  'untuk',
  'menguji',
  'kamu."',
  'Atau',
  'Allah',
  'menciptakan',
  'kematian',
  'dan',
  'kehidupan',
  'untuk',
  'menguji',
  'kamu',
  'siapa',
  'yang',
  'lebih',
  'mempersiapkan',
  'diri',
  'menghadapi',
  'kematian',
  'dan',
  'siapa',
  'yang',
  'lebih',
  'bergegas',
  'memenuhi',
  'ketaatan',
  'kepada',
  'Allah.\n'],
 ['Ada',
  'juga',
  'yang',
  'memahami',
  'mati',
  'dalam',
  'arti',
  'ketiadaan',
  'wujud.',
  'Yang',
  'memahami',
  'demikian,',
  'memahami',
  'ayat',
  'di',
  'atas',
  'dalam',
  'arti',
  'Allah',
  'menciptakan',
  'sebab-sebab',
  'kematian.',
  ''],
 ['Kematian',
  'manusia',
  'dalam',
  'pentas',
  'bumi',
  'ini',
  'bukanlah',
  'ketiadaan.'],
 ['Ia',
  'masih',
  'wujud',
  'tetapi',
  'berpindah',

In [84]:
sentence_similarity_matrix = build_similarity_matrix(full_sentences,stop_words)
sentence_similarity_matrix

array([[0.        , 0.13530202, 0.21764288, 0.1720618 , 0.19088543,
        0.        , 0.        , 0.        , 0.16222142, 0.11470787,
        0.        ],
       [0.13530202, 0.        , 0.        , 0.14744196, 0.16357216,
        0.        , 0.        , 0.04445542, 0.0695048 , 0.        ,
        0.04662524],
       [0.21764288, 0.        , 0.        , 0.        , 0.0877058 ,
        0.        , 0.        , 0.        , 0.4472136 , 0.        ,
        0.        ],
       [0.1720618 , 0.14744196, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.19088543, 0.16357216, 0.0877058 , 0.        , 0.        ,
        0.        , 0.        , 0.0836242 , 0.06537205, 0.06933752,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.

In [88]:
# page rank alogrithm
sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
scores = nx.pagerank(sentence_similarity_graph)
scores

{0: 0.18213945171093796,
 1: 0.12476155611594497,
 2: 0.12911693277786843,
 3: 0.06737252352476578,
 4: 0.13104817539726937,
 5: 0.014778325123152712,
 6: 0.04191877783947296,
 7: 0.03664925126244348,
 8: 0.1284632557796508,
 9: 0.06786365075012174,
 10: 0.07588809971837199}

In [89]:
ranked_sentence = sorted(((scores[i],list_file[i],s) for i,s in enumerate(full_sentences)), reverse=True)    
print("Indexes of top ranked_sentence order are ", ranked_sentence)

Indexes of top ranked_sentence order are  [(0.18213945171093796, '67-2-10.txt', ['Ada', 'juga', 'yang', 'memahaminya', 'dalam', 'arti:', '“Allah', 'menciptakan', 'kematian', 'untuk', 'membangkitkan', 'dan', 'memberi', 'kamu', 'balasan', 'dan', 'menciptakan', 'kehidupan', 'untuk', 'menguji', 'kamu."', 'Atau', 'Allah', 'menciptakan', 'kematian', 'dan', 'kehidupan', 'untuk', 'menguji', 'kamu', 'siapa', 'yang', 'lebih', 'mempersiapkan', 'diri', 'menghadapi', 'kematian', 'dan', 'siapa', 'yang', 'lebih', 'bergegas', 'memenuhi', 'ketaatan', 'kepada', 'Allah.\n']), (0.13104817539726937, '67-2-9.txt', ['Ujian', 'menyangkut', 'hidup', 'dan', 'mati', 'dipahami', 'oleh', 'sementara', 'ulama', 'dalam', 'arti', 'musibah', 'kematian', 'yang', 'menimpa', 'keluarga', 'atau', 'teman', 'seseorang,', 'demikian', 'juga', 'anugerah', 'kehidupan', 'serta', 'kelahiran', 'merupakan', 'bahan', 'ujian', 'Allah', 'kepada', 'manusia,', 'apakah', 'dia', 'tabah', 'dan', 'sabar', 'serta', 'bersyukur', 'dan', 'berteri

In [90]:
ranked_sentence

[(0.18213945171093796,
  '67-2-10.txt',
  ['Ada',
   'juga',
   'yang',
   'memahaminya',
   'dalam',
   'arti:',
   '“Allah',
   'menciptakan',
   'kematian',
   'untuk',
   'membangkitkan',
   'dan',
   'memberi',
   'kamu',
   'balasan',
   'dan',
   'menciptakan',
   'kehidupan',
   'untuk',
   'menguji',
   'kamu."',
   'Atau',
   'Allah',
   'menciptakan',
   'kematian',
   'dan',
   'kehidupan',
   'untuk',
   'menguji',
   'kamu',
   'siapa',
   'yang',
   'lebih',
   'mempersiapkan',
   'diri',
   'menghadapi',
   'kematian',
   'dan',
   'siapa',
   'yang',
   'lebih',
   'bergegas',
   'memenuhi',
   'ketaatan',
   'kepada',
   'Allah.\n']),
 (0.13104817539726937,
  '67-2-9.txt',
  ['Ujian',
   'menyangkut',
   'hidup',
   'dan',
   'mati',
   'dipahami',
   'oleh',
   'sementara',
   'ulama',
   'dalam',
   'arti',
   'musibah',
   'kematian',
   'yang',
   'menimpa',
   'keluarga',
   'atau',
   'teman',
   'seseorang,',
   'demikian',
   'juga',
   'anugerah',
   'kehidup

In [91]:
#top_n = int(len(scores)/2)
top_n = 7
summarize_text = []
for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][2]))

In [92]:
summarize_text

['Ada juga yang memahaminya dalam arti: “Allah menciptakan kematian untuk membangkitkan dan memberi kamu balasan dan menciptakan kehidupan untuk menguji kamu." Atau Allah menciptakan kematian dan kehidupan untuk menguji kamu siapa yang lebih mempersiapkan diri menghadapi kematian dan siapa yang lebih bergegas memenuhi ketaatan kepada Allah.\n',
 'Ujian menyangkut hidup dan mati dipahami oleh sementara ulama dalam arti musibah kematian yang menimpa keluarga atau teman seseorang, demikian juga anugerah kehidupan serta kelahiran merupakan bahan ujian Allah kepada manusia, apakah dia tabah dan sabar serta bersyukur dan berterima kasih.',
 'Kematian manusia dalam pentas bumi ini bukanlah ketiadaan.',
 'Hemat penulis, kalaupun kematian diartikan dengan ketiadaan, itu hanya berarti ketiadaan di pentas bumi ini.\n',
 'Ada juga yang memahami mati dalam arti ketiadaan wujud. Yang memahami demikian, memahami ayat di atas dalam arti Allah menciptakan sebab-sebab kematian. ',
 'Kata aula ditujukan 

In [93]:
str_summarize = ' '.join(summarize_text)
print("The top {} ranking : \n{}".format(top_n, str_summarize))

The top 7 ranking : 
Ada juga yang memahaminya dalam arti: “Allah menciptakan kematian untuk membangkitkan dan memberi kamu balasan dan menciptakan kehidupan untuk menguji kamu." Atau Allah menciptakan kematian dan kehidupan untuk menguji kamu siapa yang lebih mempersiapkan diri menghadapi kematian dan siapa yang lebih bergegas memenuhi ketaatan kepada Allah.
 Ujian menyangkut hidup dan mati dipahami oleh sementara ulama dalam arti musibah kematian yang menimpa keluarga atau teman seseorang, demikian juga anugerah kehidupan serta kelahiran merupakan bahan ujian Allah kepada manusia, apakah dia tabah dan sabar serta bersyukur dan berterima kasih. Kematian manusia dalam pentas bumi ini bukanlah ketiadaan. Hemat penulis, kalaupun kematian diartikan dengan ketiadaan, itu hanya berarti ketiadaan di pentas bumi ini.
 Ada juga yang memahami mati dalam arti ketiadaan wujud. Yang memahami demikian, memahami ayat di atas dalam arti Allah menciptakan sebab-sebab kematian.  Kata aula ditujukan kep

## Evaluation

In [101]:
# calculate f-score
def get_fscore(result_sentences, target_sentences):
    """
    Precision(P) is the number of sentences occurring in both system and ideal summaries divided by the number of sentences in the system summary
    Recall(R) is the numberof sentences occurring in both system and ideal summaries divided by the number ofsentences in the ideal summary
    """
    # get number of sentences
    num_res_sentences = len(result_sentences)
    print("num_res_sentences : {}".format(num_res_sentences))
    num_target_sentences = len(target_sentences)
    print("num_target_sentences : {}".format(num_target_sentences))
    
    # find same sentence 
    num_occur_sentences = 0
    for res_sentence in result_sentences:
        #print("res_sentence : {}".format(res_sentence))
        
        # check res_sentence appear in target sentence
        if res_sentence in target_sentences:
            #print("Find...")
            num_occur_sentences += 1 # increment
            
    print("num_occur_sentences : {}".format(num_occur_sentences))
    
    try:
        precision = num_occur_sentences / num_res_sentences
        print("precision : {}".format(precision))
        
        recall = num_occur_sentences / num_target_sentences
        print("recall : {}".format(recall))
        
        f_score = 2 * (precision * recall) / (precision + recall)
        print("f_score : {}".format(f_score))
        
        return precision, recall, f_score
    
    except ZeroDivisionError:
        return 0 , 0 , 0


In [102]:
# file list in folder
folder_root = "data set"
sub_folder = "Target Summarization"
target_file_name = os.path.join(folder_root,sub_folder,target_file) # join folder root and label
file = open(target_file_name, "r")
target_summarization = file.read()
target_summarization

'Kata hasad adalah iri hati atas nikmat disertai dengan harapan kiranya nikmat itu hilang darinya, baik diperoleh oleh yang dimiliki orang lain yang iri maupun tidak. Bahkan, sementara ulama memperluas arti hasad/ iri hati sehingga tidak hanya mencakup kedengkian terhadap pihak lain yang memiliki atau diduga memiliki nikmat, tetapi juga yang tidak memiliki nikmat apa-apa, namun kedengkian kepadanya mengantar yang dengki untuk menginginkan agar yang bersangkutan terus-menerus berada dalam kekurangan dan kepedihannya. Kata hasad digunakan juga dalam arti keinginan memeroleh nikmat dimiliki orang serupa dengan yang nikmat yang diperoleh orang lain itu. Ini biasa juga dinamai ghibthah. Sementara pakar menyatakan bahwa penyebab hasad/ iri hati antara lain adalah Keangkuhan sehingga merasa bahwa wajar untuk yang bersangkutan tetapi itu hanya wajar untuk dirinya sendiri. Penyebab hasad / iri hati yang kedua ialah persaingan, khususnya dalam bidang materi. Di sini, iri hati muncul akibat ketid

In [103]:
result_sentences = sent_tokenize(str_summarize)
result_sentences

['Hasad/ Iri hati ini dapat juga tertuju kepada orang yang sebenarnya tidak memiliki nikmat, namun diduga oleh yang iri memilikinya.',
 'Kata hasad adalah iri hati atas nikmat disertai dengan harapan kiranya nikmat itu hilang darinya, baik diperoleh oleh yang dimiliki orang lain yang iri maupun tidak.',
 'Penyebab hasad / iri hati yang kedua ialah persaingan, khususnya dalam bidang materi.',
 'Di sini, iri hati muncul akibat ketidakmampuan menyamai atau melebihi orang lain yang disaingi apa yang dimiliki.',
 'Penyebab hasad / iri hati selanjutnya adalah rasa takut dan cinta kekuasaan.',
 'Watak buruk yang telah menjadi sifat seseorang karena watak ini mengantar yang bersangkutan iri terhadap orang lain tanpa suatu sebab.',
 'Inilah jenis penyebab hasad / iri hati yang terburuk.',
 'Bila dianalisis lebih jauh, dapat disimpulkan bahwa sumber utama dari iri hati bahkan semua sifat-sifat tercela adalah perasaan rendah diri yang tersembunyi di dalam jiwa seseorang.',
 'Rasa tersebut lahir k

In [104]:
# split it to token
target_sentences = sent_tokenize(target_summarization)
target_sentences

['Kata hasad adalah iri hati atas nikmat disertai dengan harapan kiranya nikmat itu hilang darinya, baik diperoleh oleh yang dimiliki orang lain yang iri maupun tidak.',
 'Bahkan, sementara ulama memperluas arti hasad/ iri hati sehingga tidak hanya mencakup kedengkian terhadap pihak lain yang memiliki atau diduga memiliki nikmat, tetapi juga yang tidak memiliki nikmat apa-apa, namun kedengkian kepadanya mengantar yang dengki untuk menginginkan agar yang bersangkutan terus-menerus berada dalam kekurangan dan kepedihannya.',
 'Kata hasad digunakan juga dalam arti keinginan memeroleh nikmat dimiliki orang serupa dengan yang nikmat yang diperoleh orang lain itu.',
 'Ini biasa juga dinamai ghibthah.',
 'Sementara pakar menyatakan bahwa penyebab hasad/ iri hati antara lain adalah Keangkuhan sehingga merasa bahwa wajar untuk yang bersangkutan tetapi itu hanya wajar untuk dirinya sendiri.',
 'Penyebab hasad / iri hati yang kedua ialah persaingan, khususnya dalam bidang materi.',
 'Di sini, iri

In [105]:
# get precision, recall, f_score
precision, recall, f_score = get_fscore(result_sentences, target_sentences)

num_res_sentences : 18
num_target_sentences : 19
num_occur_sentences : 15
precision : 0.8333333333333334
recall : 0.7894736842105263
f_score : 0.8108108108108109
